In [ ]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment
from defdap.quat import Quat

from pathlib import Path

import copy 
import pandas as pd
import datetime

from scipy.signal import find_peaks
from scipy.linalg import sqrtm, polar


import os

# get dictools stuff 
import sys
sys.path.append("c:/work/hrdic-tools/")
import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

In [ ]:
exp = experiment.Experiment()

# load DIC data 
data_dir = Path('.')
dic_frame = experiment.Frame()
for dic_file in sorted(data_dir.glob('map_*.txt')):
    hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')

In [ ]:
ebsd_frame = experiment.Frame()
ebsd.Map(data_dir / 'Post_EBSD/map_small.cpr',
         increment=exp.increments[0], frame=ebsd_frame)

In [ ]:
ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.set_homog_point()

dic_map = exp.increments[0].maps['hrdic']
dic_map.set_homog_point(vmin=0,vmax=0.1)

In [ ]:
dic_frame.s

In [ ]:
ebsd_frame.homog_points= [(275, 379),
 (300, 199),
 (655, 232),
 (675, 124),
 (891, 264),
 (1166, 289),
 (1481, 211),
 (1404, 592),
 (959, 756),
 (698, 977),
 (1279, 949),
 (1433, 1026),
 (1614, 1404),
 (984, 1502),
 (1084, 1192),
 (875, 1103),
 (337, 1258),
 (546, 1448),
 (381, 1108),
 (507, 748),
 (451, 549)]

In [ ]:
dic_frame.homog_points = [(184, 460),
 (214, 205),
 (656, 240),
 (683, 93),
 (957, 266),
 (1306, 289),
 (1697, 167),
 (1607, 684),
 (1052, 925),
 (727, 1220),
 (1450, 1180),
 (1641, 1268),
 (1862, 1763),
 (1086, 1885),
 (1208, 1481),
 (950, 1374),
 (290, 1604),
 (543, 1830),
 (334, 1401),
 (484, 925),
 (402, 665)]

In [ ]:
hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth

for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.link_ebsd_map(ebsd_map, transform_type="projective")
    


    dic_map.set_scale(pixelsize)
    dic_map.set_crop(left=100,right=100,top=100,bottom=100)

    dic_map.plot_map(
        'max_shear', vmin=0, vmax=0.10, 
        plot_scale_bar=False, plot_gbs='line'
    )
    plt.tight_layout()

In [ ]:
dic_map.data.keys()

In [ ]:
f = dic_map.data['f']
r = np.zeros(f.shape)

for i in range(0,f.shape[2]):
    for j in range(0,f.shape[3]):
        r[:,:,i,j],_ = polar(f[:,:,i,j])

# force this into defdap! 
dic_map.data['r'] = r

In [ ]:
# test with a single map to make things easier
# exp = experiment.Experiment()

dic_dir = Path('./DIC/')
ebsd_pre_dir = Path('./Pre_EBSD/')
ebsd_post_dir = Path('./Post_EBSD/')

dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
dic_map = hrdic.Map(dic_dir/ 'map_44.txt', data_type='openpiv')

hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth


# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)

dic_map.set_scale(pixelsize)
dic_map.set_crop(left=100,right=100,top=100,bottom=100)

ebsd_pre_frame = experiment.Frame()
ebsd_map=ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

ebsd_map.find_grains()
ebsd_map.build_neighbour_network()

In [ ]:
# exp = experiment.Experiment()

# dic_dir = Path('./DIC/')
# ebsd_pre_dir = Path('./Pre_EBSD/')
# ebsd_post_dir = Path('./Post_EBSD/')

# dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
# hfw = 20.0 # microns
# pixelwidth = 2048
# pixelsize = hfw/pixelwidth

# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)
    
# ebsd_pre_frame = experiment.Frame()
# ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

# ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.find_boundaries()
# ebsd_map.find_grains
# ebsd_map.build_neighbour_network()

In [ ]:
# ebsd_post_frame = experiment.Frame()
# ebsd.Map(ebsd_post_dir / 'map_small.cpr',increment=exp.increments[-1], frame=ebsd_post_frame)

In [ ]:
exp.increments[1].maps['ebsd']

In [ ]:
# dic_map = exp.increments[0].maps['hrdic']
dic_map.set_homog_point(vmin=0,vmax=0.2)

# ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.set_homog_point()

# dic_map.homog_points = [(1722, 1177),
#  (1335, 1313),
#  (948, 1745),
#  (549, 1661),
#  (245, 1680),
#  (233, 1300),
#  (579, 1158),
#  (443, 756),
#  (559, 136),
#  (83, 359),
#  (1438, 495),
#  (857, 165)]

# ebsd_map.homog_points = [(2541, 1823),
#  (2240, 1932),
#  (1940, 2266),
#  (1631, 2207),
#  (1394, 2225),
#  (1388, 1933),
#  (1659, 1820),
#  (1554, 1519),
#  (1648, 1054),
#  (1279, 1219),
#  (2323, 1319),
#  (1875, 1080)]

# dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
# dic_map.plot_map('max_shear', vmin=0,vmax=0.1,plot_scale_bar=True,plot_gbs=True)
#  # link maps
# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_crop
#     dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
#     # dic_map.plot_map(
#     #     'max_shear', vmin=0, vmax=0.10, 
#     #     plot_scale_bar=False, plot_gbs='line'
#     # )
    # plt.tight_layout()

In [ ]:
# find twin boundaries
misori_twin = Quat.from_axis_angle([1, 1, 1], 60*np.pi/180)
misori_twin_tol = 5*np.pi/180

# create all symmetric equivalent misorientations
misori_twin_all = []
syms = ebsd_map.primary_phase.crystal_structure.symmetries
for sym_i in syms:
    for sym_j in syms:
        misori_twin_all.append(sym_i.conjugate * misori_twin * sym_j)
# get rid of any duplicates
misori_twin_all = list(set(misori_twin_all))

# loop over all grain boundary segments and check if the misorientation between
# the two grains is within tolerance of the twin misorientation
twin_lines = []
for grain1, grain2, b_seg in ebsd_map.neighbour_network.edges.data('boundary'):
    twin = False
    misori = grain2.ref_ori * grain1.ref_ori.conjugate
    for misori_twin in misori_twin_all:
        if 2 * np.arccos(misori_twin.dot(misori)) < misori_twin_tol:
            twin = True
            break
    
    if not twin:
        continue
    
    twin_lines += b_seg.boundary_lines

In [ ]:
ebsd_map.build_neighbour_network()

In [ ]:
ebsd_map.

In [ ]:
t = ebsd_map.neighbour_network.edges.data('boundary')

In [ ]:
plot = ebsd_map.plot_ipf_map([1,0,0])
plot.add_grain_boundaries(kind='line',boundaries=twin_lines, colour='red')

In [ ]:
 # plot maps
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_crop
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    # dic_map.plot_map(
    #     'max_shear', vmin=0, vmax=0.10, 
    #     plot_scale_bar=False, plot_gbs='line'
    # )
    # plt.tight_layout()

dic_map = exp.increments[-1].maps['hrdic'] 
dic_map.plot_map(
    
)